In [41]:
# ===== TASK ONE ====
import requests
import pandas as pd
import io

# =========== EXPLORE THE DATA FIRSTLY (CODE FROM THE WEBSITE) ============
# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# online_retail = fetch_ucirepo(id=352) 
  
# # data (as pandas dataframes) 
# X = online_retail.data.features 
# y = online_retail.data.targets 
  
# # metadata 
# print(online_retail.metadata) 
  
# # variable information 
# print(online_retail.variables) 

# =========== CREATE FUNCTION ============
def download_dataset():

    data_url = "https://archive.ics.uci.edu/static/public/352/data.csv"
    response = requests.get(data_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the content of the response to a pandas DataFrame
        df = pd.read_csv(io.StringIO(response.text))
        print("Dataset downloaded and stored in a DataFrame successfully!")
    else:
        print(f"Failed to download the dataset. Status code: {response.status_code}")

    # Store it into the designated data directory
    df.to_csv("data/train.csv", index = False)

download_dataset()

Dataset downloaded and stored in a DataFrame successfully!


In [52]:
df = pd.read_csv("data/train.csv")

In [43]:
# ===== TASK TWO ====
# ===== HANDLE MISSING VALUES ====
# Find and display missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Description can be filled, but when it comes to customerid, let's have a look if the same invoice
# has more customerid values so we could maybe fill it like that
invoice_customer_check = df.groupby('InvoiceNo')['CustomerID'].nunique()
discrepancies = invoice_customer_check[invoice_customer_check > 1]
print("Invoices with different Customer IDs:")
# it doesn't so we can't fill customerid

# Display rows with missing values
rows_with_missing_values = df[df.isnull().any(axis=1)]
print("Rows with missing values:")
print(rows_with_missing_values)

# Fill missing values in Description based on StockCode
stockcode_description_map = (df.dropna(subset=['Description'])
                             .drop_duplicates('StockCode')[['StockCode', 'Description']]
                             .set_index('StockCode')['Description']
                             .to_dict())

df['Description'] = df['Description'].fillna(df['StockCode'].map(stockcode_description_map))
print("Missing values in Description column filled successfully!")

# ===== CONVERT DATA TYPES ====
# df.info()
# Using the variables table in the website we see most of the variables as converted correctly.
# so InvoiceNo is object which we agreed should be an object and not numeric, since in the table it's categorical
# only variable to change is InvoiceDate and CustomerID
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')
df['CustomerID'] = df['CustomerID'].astype('float').astype('Int64').astype(str)
# ===== REMOVE DUPLICATES VALUES ====
df = df.drop_duplicates().reset_index(drop = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB
